# Problem Description
Wake county located in North Carolina is one of the fastest growing counties in the USA large influx of immigrants from other states and nationalities calling it their home.
The highly educated workforce with a good mix of young and middle aged population drives the economy of the state and opens up tremendous  possibilities for new businesses to thrive.North carolina has been influenced greatly by settlers from New England and Hispanic immigrants.North carolinais also famous for its barbecue and bacon but other cuisines are starting to becoming popular.
This opens up a possibility to create a recommender system for anyone who wants to open a new restaurant that can be successful in the area based on the information
1. Demographics and Income
2. Existing restaurants density in the area(To identify any untapped market)
3. Types of restaurants based on cuisines
4. Existing Restaurant quality based on reviews and food inspections
5. Farmer's Market around the various neighbourhood

Wake county has 12 Towns and  46 Communities and for this work we will obtain the following datasets from https://data-wake.opendata.arcgis.com/datasets
1. Wake county restaurants Datasets(csv)
2. Wake county food inspections datasets(csv)
3. Wake county demographic information(csv)
4. Farmers Market http://www.wakegov.com/humanservices/publichealth/information/promotion/Pages/farmersmarkets.aspx
The data from the above datasets will be read via pandas dataframe and cleaned up to classify Restaurants to Townships and and quality index score.Then we will be able to cluster the restaurants in the area and as well as correlate the quality of restaurants again based on cuisine to the neighbourhood which would potentially provide us insights on the type of restaurant that can be setup. We will be using Foursquare api to obtain restaurant reviews in the locality to do additional correlation that will strengthen the model.We will finally demonstrate the efficacy with vuisualisation plots and graphs that will aid in the final decision

## Recommendation Success Criteria
We will determine the success of the recommender system by providing potential locations it can be set up for maximum success